# PPD: MPI e programação com passagem de mensagem

Hélio - DC/UFSCar - 2023

# Comunicações coletivas

Algumas formas de comunicação oferecidas por MPI são operações coletivas (*Collective communications*) e envolvem **todos** os processos pertencentes a um grupo (*communicator*) de uma só vez. 

Nessas operações, a mesma chamada deve ser emitida por todos os processos do grupo envolvido, sendo que um ou mais processos irão enviar dados e um ou mais irão recebê-los. Cabe ao programador, contudo, garantir que todos os processos do grupo emitam as chamadas apropriadas. 

Tipos de operações coletivas (*Collective Operations*):

* ***Synchronization***: operações coletivas deste tipo servem para que processos esperem até que todos os membros do grupo atinjam um ponto de sincronização;
* ***Data Movement***: permitem a transferência de dados, em diferentes modelos (*broadcast, scatter/gather, all to all, ...*);
* ***Collective Computation (reductions)***: são operações coletivas em que um membro do grupo coleta os dados e realiza uma operação sobre eles, como nas operações de redução (min, max, add, multiply, etc.)

Considerações:

* Como envolvem todos os processos em um grupo (*communicator*) operações coletivas são bloqueantes e têm o efeito de sincronizar todos os processos do grupo.
* Não há *tags* nas mensagens envolvidas nessas operações.
* Para realizar operações coletivas usando apenas sub-conjuntos dos processos da aplicação, é preciso antes criar os sub-grupos (*communicators*) apropriados.
* Apenas tipos pré-definidos por MPI podem ser usados nas comunicações coletivas.

Primitivas para comunicações coletivas:

* **MPI_Barrier** (comm): Cria uma barreira de sincronização para os processo de um grupo. Todos os processos que realizam a chamada são bloqueados até que o último processo do grupo faça esta mesma chamada. 
* **MPI_Bcast** (&buffer,count,datatype,root,comm): o processo cujo rank for especificado como *root* na chamada difunde (*broadcasts*) uma mensagem para todos os demais do grupo.
* **MPI_Scatter** (&sendbuf,sendcnt,sendtype,&recvbuf,...... recvcnt,recvtype,root,comm): o processo identificado como *root* na chamada distribui partes distintas da mensagem para os demais processos do grupo. 
* **MPI_Scatterv** ( ): 
* **MPI_Gather** (&sendbuf,sendcnt,sendtype,&recvbuf, ...... recvcount,recvtype,root,comm): de forma contrária ao que ocorre em *MPI_Scatter*, nesta chamada, o processo identificado como *root* coleta (*gathers*) mensagens distintas de todos os demais nós do grupo, concatenando-as em posições distintas do buffer de recepção, de acordo com o número lógico dos processos emissores.
* **MPI_Gatherv**( ):
* **MPI_Allgather** (&sendbuf,sendcount,sendtype,&recvbuf, ...... recvcount,recvtype,comm): nessa operaçao de concatenação, todos os processos do grupo obtêm os dados transmitidos por todos no grupo.

Nas operações coletivas de **redução**, além de coletar os dados enviados por todos os processos do grupo, o processo *root* realiza uma operação de agregação sobre os dados.

* **MPI_Reduce** (&sendbuf,&recvbuf,count,datatype,op,root,comm): processo *root* coleta todos os dados e aplica uma operação de redução.

* **MPI_Allreduce** (const void *sendbuf, void *recvbuf, int count, MPI_Datatype datatype, MPI_Op op, MPI_Comm comm): também realiza a redução, mas todos os processos do grupo têm o resultado.

* **MPI_Reduce_scatter** (&sendbuf,&recvbuf,recvcount,datatype, ...... op,comm): *First does an element-wise reduction on a vector across all tasks in the group. Next, the result vector is split into disjoint segments and distributed across the tasks. This is equivalent to an MPI_Reduce followed by an MPI_Scatter operation*.

* **MPI_Alltoall** (&sendbuf,sendcount,sendtype,&recvbuf, ...... recvcnt,recvtype,comm): *Each task in a group performs a scatter operation, sending a distinct message to all the tasks in the group in order by index*.

* **[MPI_Scan](https://www.open-mpi.org/doc/v3.1/man3/MPI_Scan.3.php)**(&sendbuf,&recvbuf,count,datatype,op,comm): *Performs a scan operation with respect to a reduction operation across a task group*.

```
MPI Reduction   Operation                 C Data Types
MPI_MAX         maximum                   integer, float
MPI_MIN         minimum                   integer, float
MPI_SUM         sum                       integer, float
MPI_PROD        product                   integer, float
MPI_LAND        logical AND               integer
MPI_BAND        bit-wise AND              integer
MPI_LOR         logical OR                integer 
MPI_BOR         bit-wise OR               integer, MPI_BYTE
MPI_LXOR        logical XOR               integer
MPI_BXOR        bit-wise XOR              integer, MPI_BYTE
MPI_MAXLOC      max value and location    float, double and long double
MPI_MINLOC      min value and location    float, double and long double
```


# Broadcast

Ao criar aplicações que se comunicam via MPI, independentemente da tecnologia da rede física e dos protocolos usados na interligação dos computadores,  não é preciso preocupar-se com detalhes das transmissões. Não é preciso ficar cuidando de quais são os processos que compõem a aplicação, de quais são os endereços IP dos computadores em que esses processos estão sendo executados, dos números de porta utilizados, de detalhes de *buffers* e retransmissões. Tudo isso é feito pela implementação MPI.

 Por exemplo, nos casos em que é preciso **enviar a mesma informação** para **todos** os processos da aplicação (que estão no mesmo MPI_COMM_WORLD), há 2 formas: replicando envios individuais, ou usando [MPI_Bcast](https://www.open-mpi.org/doc/v3.1/man3/MPI_Bcast.3.php).
```
1. // Broadcast com envios individuais
  int root = 0;
  ...
  MPI_Comm_size(MPI_COMM_WORLD,&num_tasks);

  if (rank == root)
    for (i = 1; i < num_taks; i++) {
      // int MPI_Send(void *buf, int count, MPI_Datatype dtype, int dest,
      //              int tag, MPI_Comm comm)
      MPI_Send(tx_buf, count, data_type, i, tag, MPI_COMM_WORLD);
  else
    // int MPI_Recv(void *buf, int count, MPI_Datatype dtype, int src, 
    //              int tag, MPI_Comm comm, MPI_Status *stat)
    MPI_Recv(rx_buf, count, data_type, root, tag, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
```
No caso 1, a aplicação identificou quantos processos estão sendo usados nesta execução e realizou envios individuais, replicando a mesma mensagem para todos. Nas transmissões, o índice do comando de iteração (***i***) foi usado para identificar o destino de cada mensagem, variando de ***1 a num_taks -1***.

Todos os demais processos, com ***rank*** **> 0**, executaram uma operação de recebimento correspondente às mensagens lhes enviadas pelo proceso de *rank* **0** .

<br>

A segunda forma de enviar um mesmo conteúdo para todos os processos de um grupo é o uso da operação coletiva de ***broadcast***, ilustrado a seguir.

```
2.  // TODOS os processos do grupo (MPI_COMM_WORLD, neste caso), 
    // independentemente de seus ranks, têm que fazer a chamada.

    // O processo root tem os dados no buf. 
    // Nos demais, buf é o endereço onde os dados recebidos serão copiados. 

    // int MPI_Bcast(void *buffer, int count, MPI_Datatype datatype, int root, MPI_Comm comm) 
    MPI_Bcast( buf, count, data_type, root, MPI_COMM_WORLD);
```
No caso 2, a aplicação realiza a difusão do conteúdo do *buffer* usando uma única chamada de *broadcast*. Nesta operação, **um** dos processos do grupo vai fazer a **transmissão** e **todos os demais** membros deste grupo vão **recebê-la**. 

Se o grupo especificado na chamada for MPI_COMM_WOLRD, isso significa que todos os processos da aplicação vão ter que executar a mesma chamada.

Nesta chamada, o processo emissor é identificado pelo ***rank*** informado no parâmetro ***root***.  

<br>

Como todos os processos do grupo realizam a mesma chamada, é preciso atentar para o endereço passado no parâmetro *buffer*. Para o nó *root*, que é o emissor, o *buffer* aponta para o endereço de onde os dados a serem transmitidos estão posicionados na memória deste processo. Para os demais processos, passa-se o endereço da posição de memória **onde eles serão copiados**. 

Se o modelo de execução SMPD for usado na ativação do programa MPI, em que o mesmo código é executado em todos os processos, é preciso diferenciar os papeis que os processos com os diferentes *ranks* irão realizar.

<br>

Será que há alguma vantagem em usar o caso 2? Isso depende da tecnologia de transmissão usada na rede (Ethernet, e.g.).

É claro que, nesta forma de transmissão, cabe à **implementação MPI** fazer com que a mesnagem seja efetivamente enviada a todos os processos do grupo. Se isso será feito com UDP ou TCP, usando *Ethernet broadcast*, ou mensagens replicadas, (felizmente :-) não é problema da aplicação. 

<br>

Vejamos um exemplo a seguir.

In [ ]:
%%writefile bcast.c

#include <sys/types.h>
#include <unistd.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <mpi.h>

#define N 10

#define root 0

int A[N], B[N], C[N];

int
main( int argc, char *argv[])
{
	int i, rank, result, numtasks;

	result = MPI_Init(&argc,&argv);

	if (result != MPI_SUCCESS) {
		printf ("Erro iniciando programa MPI.\n");
		MPI_Abort(MPI_COMM_WORLD, result);
	}

	MPI_Comm_size(MPI_COMM_WORLD,&numtasks);
	MPI_Comm_rank(MPI_COMM_WORLD,&rank);

  // inicia semente do gerador de números aleatórios
	// srand(time(NULL));
  srand(getpid());
  
	// Todos iniciam vetor A
	for (i=0; i < N; i++)
		A[i]=rand() % 10;

	// Processo com rank 0 foi escolhido para gerar vetor B e propagá-lo aos demais
	if(rank==root) 
		for (i=0; i < N; i++)
			B[i]=rand() % 10;

	// Todos os processos participam do broacast. 
	// E claro que essa chamada poderia estar em trechos distintos do codigo, 
  // usando buffers distintos.
	// O que importa é que todos facam a chamada e, exceto pelo root, 
  // tenham espaco para armazenar os dados.
	// O processo root, 0 neste caso, tem o vetor cujo conteúdo sejá enviado
 
	MPI_Bcast( B, N, MPI_INT, root, MPI_COMM_WORLD); 

  printf("(%d) B: ",rank);
	for(i=0; i < N; i++) 
		printf("%d ",B[i]);
	printf("\n"); fflush(stdout);
 
	// todos fazem a operacao usando o mesmo vetor B
	for(i=0;i<N;i++)
		C[i]=A[i]+B[i];
	// ...

	// todos imprimem o vetor C. 
	// Na ativaçãodos processos remotos com mpirun, stdout é redirecionado para
	// o nó de origem. Assim, todas as impressões vão aparecer no terminal. 		
  printf("(%d) C: ",rank);
	for(i=0; i < N; i++) 
		printf("%d ",C[i]);
	printf("\n"); fflush(stdout);

	MPI_Finalize();

	return(0);
}

In [ ]:
# ! if [ ! bcast -nt bcast.c ]; then mpiicc -Wall bcast.c -o bcast; fi && mpirun -n 4 -host localhost:4 bcast
# !./compile.sh bcast 
# !./launch.sh bcast

# Scatter & Gatther

Dentre as operações MPI para comunicação coletiva, a primitiva [MPI_Scatter](https://www.open-mpi.org/doc/v3.1/man3/MPI_Scatter.3.php) causa o **envio particionado** de uma mensagem a **todos** os processos de um grupo (*communicator*). 

```
int MPI_Scatter(const void *sendbuf, int sendcount, MPI_Datatype sendtype,
                void *recvbuf, int recvcount, MPI_Datatype recvtype, 
                int root, MPI_Comm comm);
int MPI_Iscatter(const void *sendbuf, int sendcount, MPI_Datatype sendtype,
                 void *recvbuf, int recvcount, MPI_Datatype recvtype, 
                 int root, MPI_Comm comm, MPI_Request *request);
```
O efeito dessa primitiva, que faz o oposto de MPI_Gather, é como se uma série de transmissões fossem feitas com MPI_Send, cada uma enviando uma parte sequencial do conteúdo do *buffer* de transmissão para um processo separado: 
```
MPI_Send(sendbuf + i * sendcount * sizeof(sendtype), sendcount, sendtype, i, ...);
```
com cada processo (incluse o emissor) executando uma operação de recebimento equivalente:
``` 
MPI_Recv(recvbuf, recvcount, recvtype, i, ...).
```

O resultado é que cada processo do grupo vai **receber uma fração** da mensagem original, contendo um número de elementos **proporcional** à divisão do número total pelo número de processos do grupo. É como se a mensagem original fosse particionada e o **segemento** ***i*** fosse enviado ao processo ***i***.

Examinando os campos desta primitiva, vê-se que há um *buffer* de envio e que há também indicação de um processo (***rank***) que será o ***root***, ou seja, o emissor nesta transmissão. No processamento da chamada, o *buffer* de envio (***sendbuf***) é ignorado em todos os nós cujo *rank* for diferente do ***root***.

Outroa campos da primitiva incluem um ponteiro para o *buffer* de recebimento. Esse campo deverá ser válido **em todos** os processos (*ranks*), pois todos irão receber parte dos dados. O nó emissor também recebe sua fração dos dados.

É claro que os campos ***root*** e ***comm*** devem conter os mesmos valores em todos 
os nós.



# MPI_Gather

[MPI_Gather](https://www.open-mpi.org/doc/v3.1/man3/MPI_Gather.3.php) tem o efeito contrário da operação *scatter*. Assim, o resultado é que o nó *root* vai receber uma mensagem de cada processo do grupo, inclusive uma sua, e vai colocar essas mensagens em sequência no buffer de recepção, em ordem correspondente aos identificadores dos nós no grupo.

```
int MPI_Gather(const void *sendbuf, int sendcount, MPI_Datatype sendtype,
    void *recvbuf, int recvcount, MPI_Datatype recvtype, int root,
    MPI_Comm comm)
```

O programa exemplo a seguir ilustra o uso das operações de ***scatter*** e ***gather***.

In [ ]:
%%writefile scatter.c

#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

#define SIZE 10

int 
main(int argc, char *argv[])
{
	int numtasks, rank, i, j;

	int recbuf[SIZE];
	int *fulbuf;

	char hostname[MPI_MAX_PROCESSOR_NAME];
	int namelen;

	MPI_Init(&argc,&argv);

	MPI_Comm_rank(MPI_COMM_WORLD, &rank);
	MPI_Comm_size(MPI_COMM_WORLD, &numtasks);

	MPI_Get_processor_name(hostname, &namelen);

	// root node 
	if(rank == 0) {

		fulbuf = (int *)malloc(numtasks * SIZE * sizeof(int));

		// preenche vetor para envio
		for(i=0; i < numtasks; i++)
			for(j=0; j < SIZE; j++)
				fulbuf [i*SIZE + j] = i;
	}
	// Todos os processos, independentemente de seus ranks, devem emitir essa 
  // chamada de operaçào coletiva. Os dados do processo root é que serão
  // distribuídos, de forma fracionada, entre todos.
 
	// int MPI_Scatter(const void *sendbuf, int sendcount, MPI_Datatype sendtype,
	//                 void *recvbuf, int recvcount, MPI_Datatype recvtype, int root,
	//                 MPI_Comm comm);

	MPI_Scatter(fulbuf, SIZE, MPI_INT, recbuf, SIZE, MPI_INT, 0, MPI_COMM_WORLD);

	// todos recebem, inclusive o processo root
	printf("%s (%d): ", hostname, rank);
	for(i=0;i<SIZE;i++) {
		printf("%d ",recbuf[i]); 
		recbuf[i] += 10 * rank;
	}
	printf("\n"); fflush(stdout);
	
	// int MPI_Gather(const void *sendbuf, int sendcount, MPI_Datatype sendtype,
	//                void *recvbuf, int recvcount, MPI_Datatype recvtype, int root,
	//                MPI_Comm comm);
	MPI_Gather(recbuf, SIZE, MPI_INT, fulbuf, SIZE, MPI_INT, 0, MPI_COMM_WORLD);

	if(rank == 0) {
		printf("\n");
		for(i=0; i < numtasks; i++) {
			for(j=0; j<SIZE; j++)
				printf("%2d ",fulbuf[i*SIZE + j]);
			printf("\n");
		}
	}

	MPI_Finalize();
	
	return(0);
}


In [ ]:
# Aqui, testamos com 4 processos no mesmo nó. Se houver mais nós, basta configurá-los em um hostfile, ou especificar em linha de comando
!mpicc -Wall scatter.c -o scatter && mpirun -n 4 -host localhost:4 scatter

# MPI_Reduce

As operações globais de redução ([MPI_Reduce](https://www.open-mpi.org/doc/v3.1/man3/MPI_Gather.3.php), MPI_Op_create, MPI_Op_free,  MPI_Allreduce, MPI_Reduce_scatter, MPI_Scan) realizam a **sumarização global** de algum valor provido por todos os processos de um grupo.

A operação de sumarização pode ser alguma entre as operações pré-definidas (*), como a soma ou a multiplicação dos valores, a identificação do maior ou do menor valor, ou novas operações definidas pela aplicação .

```
(*) Name                Meaning
   ---------           --------------------
    MPI_MAX             maximum
    MPI_MIN             minimum
    MPI_SUM             sum
    MPI_PROD            product
    MPI_LAND            logical and
    MPI_BAND            bit-wise and
    MPI_LOR             logical or
    MPI_BOR             bit-wise or
    MPI_LXOR            logical xor
    MPI_BXOR            bit-wise xor
    MPI_MAXLOC          max value and location
    MPI_MINLOC          min value and location
```
Enquanto a operação MPI_Reduce produz o valor da redução em apenas um processo definido, [MPI_Allreduce](https://www.open-mpi.org/doc/v3.1/man3/MPI_Allreduce.3.php) produz o mesmo resultado nos *buffers* de todos os processos do grupo.

Já a chamada [MPI_Reduce_scatter](https://www.open-mpi.org/doc/v3.1/man3/MPI_Reduce_scatter.3.php) combina operação de redução com a distribuição, operando sobre um vetor de resultados combinados.

```
int MPI_Reduce(const void *sendbuf, void *recvbuf, int count,
               MPI_Datatype datatype, MPI_Op op, int root, MPI_Comm comm);
 
int MPI_Ireduce(const void *sendbuf, void *recvbuf, int count,
                MPI_Datatype datatype, MPI_Op op, int root,
                MPI_Comm comm, MPI_Request *request);
```

Entre os campos desta chamada, como era de se esperar, aparecm o endereço do ***buffer*** **de envio**, que vale para todos os nós, a indicação do processo que será o ***root*** desta operação, ou seja o ***rank*** daquele que vai receber todos os dados e sumarizá-los, a operação que será aplicada para a redução, além do tipo e a contagem dos dados.

Há algumas primitivas diferetes para redução, como descritas a seguir.

* **MPI_Reduce**(&sendbuf,&recvbuf,count,datatype,op,root,comm)
Applies a reduction operation on all tasks in the group and places the result in one task.
* **MPI_Allreduce**(const void *sendbuf, void *recvbuf, int count, MPI_Datatype datatype, MPI_Op op, MPI_Comm comm).  
Same as MPI_Reduce except that the result appears in the receive buffer of all the group members. Applies a reduction operation and places the result in all tasks in the group. This is equivalent to an MPI_Reduce followed by an MPI_Bcast.
* **MPI_Reduce_scatter**(&sendbuf,&recvbuf,recvcount,datatype, ...... op,comm). 
First does an element-wise reduction on a vector across all tasks in the group. Next, the result vector is split into disjoint segments and distributed across the tasks. This is equivalent to an MPI_Reduce followed by an MPI_Scatter operation.
* **MPI_Scan**(&sendbuf,&recvbuf,count,datatype,op,comm). 
Performs a scan operation with respect to a reduction operation across a task group.

Resumidamente, a chamada MPI_Reduce, aplica uma operação de redução sobre dados recebidos de todos os processos de um grupo.

O exemplo a seguir ilustra o uso da primitiva MPI_Reduce.


In [ ]:
%%writefile reduce.c

#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <time.h>
#include "mpi.h"


int main(int argc, char *argv[])
{
	int rank, numtasks;
  int num, redutor = 0;

	MPI_Init(&argc,&argv);

	MPI_Comm_rank(MPI_COMM_WORLD, &rank);
	MPI_Comm_size(MPI_COMM_WORLD, &numtasks);

  // srand(time(NULL));
  srand(getpid());
  num = rand() % 100;
 
	// collective communications 

	// int MPI_Reduce (void *sendbuf, void *recvbuf, int count, MPI_Datatype datatype,
	//                 MPI_Op op, int root, MPI_Comm comm);
	MPI_Reduce(&num, &redutor, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);

  // Apenas o processo de rank 0, neste caso, deve imprimir um valor consistente
  // já que ele foi o 'root' desta operação
	printf("Rank %d: num: %d  sum: %d\n", rank, num, redutor);

	// int MPI_Allreduce (void *sendbuf, void *recvbuf, int count, MPI_Datatype datatype, 
	//                    MPI_Op op, MPI_Comm comm )
	MPI_Allreduce(&num, &redutor, 1, MPI_INT, MPI_MAX, MPI_COMM_WORLD);

  // Todos os processos devem ter recebido uma cópia do valor gerado com a redução agora
	printf("Rank %d: num: %d  max: %d\n", rank, num, redutor);

	MPI_Finalize();

	return(0);
}

In [ ]:
! if [ ! reduce -nt reduce.c ]; then mpicc -Wall reduce.c -o reduce; fi && mpirun -n 4 -host localhost:4 reduce

Vejamos mais um exemplo de operação de redução.

Como se pode ver nos parâmetros da primitiva MPI_Reduce, os *buffers* de envio e recebimento podem ser um vetor, já que os demais parâmetros indicam o tipo dos dados e o número de elementos. Assim, a operação de redução pode ser aplicada a cada um dos elementos deste vetor!

Do ponto de vista de desempenho, contudo, talvez esse tipo de operação não seja muito eficiente. Embora a implementação da biblioteca MPI trate de todos os detalhes desta operação, é possível que uma solução mais eficiente seja obtida caso a redução seja feita localmente, por exemplo, usando algum algoritmo em *log_n* etapas.

In [ ]:
%%writefile red-vet.c

#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <time.h>
#include <mpi.h>
#include <string.h>

#define NELEM 10

int main(int argc, char *argv[])
{
	int i, rank, numtasks;
  int num[NELEM], redutor[NELEM];

	MPI_Init(&argc,&argv);

	MPI_Comm_rank(MPI_COMM_WORLD, &rank);
	MPI_Comm_size(MPI_COMM_WORLD, &numtasks);

  // srand(time(NULL));
  srand(getpid());
 
  // cada nó gera seus valores locais
  for (i=0; i < NELEM; i++)
    num[i] = rand() % 10;
 
  memset(redutor, 0, NELEM* sizeof(int));
 
  printf("%d: ", rank);
  for(i=0; i < NELEM; i++)
    printf("%d ",num[i]);
  printf("\n"); 
 
	// collective communications 

	// int MPI_Reduce (void *sendbuf, void *recvbuf, int count, MPI_Datatype datatype,
	//                 MPI_Op op, int root, MPI_Comm comm);
	MPI_Reduce(num, redutor, NELEM, MPI_INT, MPI_MAX, 0, MPI_COMM_WORLD);

  // Apenas o processo de rank 0, neste caso, deve imprimir um valor consistente
  // já que ele foi o 'root' desta operação
  printf("%d: ", rank);
	for (i=0; i < NELEM; i++)
    printf("%d ", redutor[i]);
  printf("\n");
 
	// int MPI_Allreduce (void *sendbuf, void *recvbuf, int count, MPI_Datatype datatype, 
	//                    MPI_Op op, MPI_Comm comm )
	MPI_Allreduce (num, redutor, NELEM, MPI_INT, MPI_SUM, MPI_COMM_WORLD);

  // Todos devem ter os valores da redução agora
  printf("%d: ", rank);
	for (i=0; i < NELEM; i++)
    printf("%d ", redutor[i]);
  printf("\n");
 
	MPI_Finalize();

	return(0);
}

In [ ]:
! if [ ! red-vet -nt red-vet.c ]; then mpicc -Wall red-vet.c -o red-vet; fi && mpirun -n 4 -host localhost:4 red-vet